In [2]:
import os
os.chdir('/Coding/CVPR2025_abaw_framewise/')

In [16]:
import os
import pandas as pd
from tqdm import tqdm
import soundfile as sf
import numpy as np

def read_meta_text(path):
    data = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(',', 2)
            video_path, label, text = parts
            file_path = os.path.join(video_path, os.path.basename(video_path).removesuffix('.mp4'))
            data.append([file_path, label, text])
    return pd.DataFrame(data, columns=['general_path', 'class_id', 'text'])

def generate_train_meta(data_folder, eval_df, jump_s=6.0):
    # Initialize an empty DataFrame with specified columns
    framewise_meta = pd.DataFrame(columns=['general_path', 'class_id', 'text', 'frame_time_index'])

    # Iterate over each row in eval_df
    for _, row in tqdm(eval_df.iterrows(), total=eval_df.shape[0], desc="Generating frame-wise metadata"):
        audio_file_path = os.path.join(data_folder, "sound_16kHz", row['general_path'] + ".wav")
        audio_data, sr = sf.read(audio_file_path, dtype="float32")
        num_frames_all = len(audio_data)
        num_frames_sample = int(((num_frames_all / sr) / jump_s))

        step_size = int(jump_s * sr)

        frame_time_indices = np.arange(0, num_frames_all, step_size)
        frame_time_indices = frame_time_indices[:num_frames_sample]

        # Append rows to framewise_meta for each timestep
        for frame_time_index in tqdm(
                frame_time_indices,  
                total=num_frames_sample, 
                desc="sub - for-loop", 
                position=1, 
                leave=False
                ):
            new_row = {
                'general_path': row['general_path'],
                'class_id': row['class_id'],
                'text': row['text'],
                'frame_time_index': frame_time_index
            }

            framewise_meta = pd.concat([framewise_meta, pd.DataFrame([new_row])], ignore_index=True)
    
    return framewise_meta

In [23]:
data_path = "data_abaw"
label_file = "data_abaw/default-split/train.txt"

jump_s = 24.0
output_csv = f"data_abaw/splits/train_jump-{jump_s:.1f}s.csv"

df = read_meta_text(label_file)
train_df = generate_train_meta(data_path, df, jump_s=jump_s)

# with open('test.txt', 'w') as file:
#     file.writelines(f"{num}\n" for num in num_frames_xl)
    
train_df.to_csv(output_csv, index=False)

Generating frame-wise metadata: 100%|██████████| 304/304 [00:01<00:00, 163.55it/s]


In [18]:
train_df.head()

,general_path,class_id,text,frame_time_index
0,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,0
1,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,24000
2,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,48000
3,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,72000
4,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,96000


In [19]:
train_df.head()

,general_path,class_id,text,frame_time_index
0,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,0
1,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,24000
2,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,48000
3,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,72000
4,82622/Visite_1/82622_Question_7_2024-11-05_15-...,1,This is a tough question. My girlfriend and I ...,96000
